In [1]:
import contextlib
import io

f = io.StringIO()
with contextlib.redirect_stdout(f):
    import numpy as np
    import pandas as pd
    from content_table import *
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

## 0. Helper Functions and set-ups

- this includes data loading and index helplers that finds the user and animation id and their indices

In [3]:
# read all the datasets
df_1 = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")

# choose a random user
# chosen_user_1 = np.random.choice(df_1['u_id'].unique(), 1)[0]
# chosen_user_2 = np.random.choice(df_2['u_id'].unique(), 1)[0]
df, pivot = load_data("data/100x100.csv")
data_df = pivot
delta=df[['u_id', 'a_id', 'score']]

user_id_index_map = {u_id: idx for idx, u_id in enumerate(data_df.index)}
anime_id_index_map = {a_id: idx for idx, a_id in enumerate(data_df.columns)}

def user_id_to_index(uid):
    return user_id_index_map.get(uid, -1)  # returns -1 if uid not found

def anime_id_to_index(aid):
    return anime_id_index_map.get(aid, -1) # returns -1 if aid not found

# choose a static user
chosen_user_1 = 966
chosen_user_2 = 966

print(f"Chosen user 1: {chosen_user_1}")
print(f"Chosen user 2: {chosen_user_2}")

Chosen user 1: 966
Chosen user 2: 966


## 1. Baseline Model

The Baseline Model aims to predict missing entries in the matrix by accounting for deviations in user and item behaviors. Specifically, it models the predicted rating as a combination of:

- the global average rating,
- the individual user bias, and
- the individual item bias.

We express the **objective function** as:

$$
\min_{R \in \mathbb{R}^{m \times n}} \sum_{(u,i) \in \Delta} (R_{ui} - A_{ui})^2 + \lambda_1 \left( \sum_{u \in U} (b_u)^2 + \sum_{i \in I} (b_i)^2 \right)
$$

Where:

- $\Delta$ is the index set of observed entries  
- $U$ is the set of all users $\{u_1, u_2, \ldots, u_m\}$  
- $I$ is the set of all animes $\{i_1, i_2, \ldots, i_n\}$  
- $A \in \mathbb{R}^{m \times n}$ is the incomplete rating matrix, where $A_{ui}$ is user $u$'s actual rating of anime $i$  
- $R \in \mathbb{R}^{m \times n}$ is the completed rating matrix, where $R_{ui}$ is user $u$'s predicted rating of anime $i$  
- $b_u$ is the deviation of user $u$'s rating from the average  
- $b_i$ is the deviation of anime $i$'s rating from the average  
- $\lambda_1$ is the regularization parameter

Optionally, we can express the predicted rating as:

$$
R_{ui} = \mu + b_u + b_i
$$

Where $\mu$ is the mean of all known ratings.

---

In summary, the first term of the objective function minimizes the squared error between the observed ratings $A_{ui}$ and the predicted ratings $R_{ui}$. The second term penalizes the sum of squared biases of users and items to avoid overfitting. The regularization parameter $\lambda_1 > 0$ discourages excessively large bias terms.

In [4]:
# baseline model lambda = 5, using dataset 1
baseline_matrix_1 = baseline_model(5, df_1)
recommend_anime(baseline_matrix_1, chosen_user_1, df_1)

,title,predicted_rating
0,Monster,8.435279
1,Steins;Gate,8.411371
2,Fullmetal Alchemist: Brotherhood,8.118812
3,Yojouhan Shinwa Taikei,8.068790
4,Ginga Eiyuu Densetsu,8.061904


In [5]:
# baseline model lambda = 5, using dataset 2
baseline_matrix_2 = baseline_model(5, df_2)
recommend_anime(baseline_matrix_2, chosen_user_2, df_2)

,title,predicted_rating
0,Gekkan Shoujo Nozaki-kun,8.816964
1,Tenkuu no Shiro Laputa,8.562032
2,Ping Pong the Animation,8.413421
3,Perfect Blue,8.363466
4,Sennen Joyuu,8.352636


## 2. $\ell_2$-Regularized Matrix Factorization Model (ALS)


Matrix factorization is another method that attempts to find a low-rank approximation of the incomplete rating matrix $A$. Specifically, the goal is to factor $A$ as the product of two low-rank matrices $X$ and $Y$:

$$
R = X Y^T
$$

Where:

- Each row $x_u$ in $X$ is the **latent feature vector** for user $u$
- Each row $y_i$ in $Y$ is the **latent feature vector** for anime item $i$

The predicted rating for user $u$ on anime $i$ is given by the dot product:

$$
R_{ui} = x_u^T y_i
$$

---

We define the **objective function** to minimize the reconstruction error on known ratings while regularizing the latent vectors:

$$
\min_{X, Y} \sum_{(u,i) \in \Delta} (A_{ui} - x_u^T y_i)^2 + \lambda_2 \left( \sum_u \|x_u\|^2 + \sum_i \|y_i\|^2 \right)
$$

This optimization problem is **non-convex**, but we use **Alternating Least Squares (ALS)** to solve it efficiently by alternating between solving for $X$ while fixing $Y$, and vice versa.

Each subproblem reduces to a regularized least squares problem:

$$
A^T A \, \mathbf{x} \approx A^T \mathbf{b}
$$

---

### ALS Update Rules

For each user $u$:

- Let $\Delta_u$ be the set of anime rated by user $u$
- Let $Y_u$ be the submatrix of $Y$ with rows $y_i$ for $i \in \Delta_u$
- Let $A_{u,\Delta_u}$ be the known ratings for those items

Then:

$$
x_u = (Y_u^T Y_u + \lambda_2 I)^{-1} Y_u^T A_{u,\Delta_u}
$$

For each anime $i$:

- Let $\Delta_i$ be the set of users who rated anime $i$
- Let $X_i$ be the submatrix of $X$ with rows $x_u$ for $u \in \Delta_i$
- Let $A_{\Delta_i,i}$ be the known ratings for those users

Then:

$$
y_i = (X_i^T X_i + \lambda_2 I)^{-1} X_i^T A_{\Delta_i,i}
$$

---

By alternating these updates, the model gradually learns low-dimensional embeddings for users and anime, producing a completed matrix $R$ that approximates the original rating matrix $A$.

In [6]:
# ALS at rank 20, 1000 iterations, using dataset 1
X, Y = als(df_1, 20, 1000)
als_matrix_1 = X @ Y.T
recommend_anime(als_matrix_1, chosen_user_1, df_1)

,title,predicted_rating
0,Clannad: After Story,9.466268
1,Ookami Kodomo no Ame to Yuki,9.094626
2,Azumanga Daioh,8.940604
3,Psycho-Pass,8.741780
4,Nichijou,8.638490


In [7]:
# ALS at rank 20, 1000 iterations, using dataset 2
X, Y = als(df_2, 20, 1000)
als_matrix_2 = X @ Y.T
recommend_anime(als_matrix_2, chosen_user_2, df_2)

,title,predicted_rating
0,Gekkan Shoujo Nozaki-kun,9.062155
1,High Score Girl,9.021654
2,Sakura Quest,8.656357
3,Mob Psycho 100 II,8.619141
4,Asobi Asobase,8.331008


## 3. Nuclear Norm Minimization Model

We also explore the classic, convex-relaxed **nuclear norm optimization** problem. This model directly completes the matrix by minimizing its nuclear norm, subject to equality constraints on known entries:

$$
\min_{R \in \mathbb{R}^{m \times n}} \left\| R \right\|_\ast \quad \text{subject to } R_{ui} = A_{ui}, \quad \forall (u,i) \in \Delta
$$

Where:

- $\left\| R \right\|_\ast$ is the **nuclear norm** of $R$, equal to the sum of its singular values.
- $\Delta$ is the set of indices $(u, i)$ for which the rating $A_{ui}$ is known.

---

Unlike spectral regularization or ALS models, this approach **does not include a reconstruction error loss term** — it assumes that the known entries are exact, and finds the **lowest-rank matrix** (in convex approximation) consistent with those observations.

This method is particularly useful when data is sparse and **low-rank structure** is expected, such as in recommendation systems.

In [8]:
# Spectral Regularization Model at lambda 5, using dataset 1
spectral_matrix_1 = spectral_regularization_model(5, df_1)
recommend_anime(spectral_matrix_1, chosen_user_1, df_1)

,title,predicted_rating
0,Monster,8.005141
1,Ouran Koukou Host Club,7.994429
2,Azumanga Daioh,7.977646
3,Psycho-Pass,7.965450
4,Higurashi no Naku Koro ni,7.942880


In [9]:
# Spectral Regularization Model at lambda 5, using dataset 2
spectral_matrix_2 = spectral_regularization_model(5, df_2)
recommend_anime(spectral_matrix_2, chosen_user_2, df_2)

,title,predicted_rating
0,Gekkan Shoujo Nozaki-kun,8.259591
1,Tenkuu no Shiro Laputa,8.178825
2,Perfect Blue,8.113580
3,Seto no Hanayome,8.047835
4,Asobi Asobase,8.030796


## 4. Spectral Regularization Model


This spectral regularization model uses the **nuclear norm** of the recovered matrix \( R \). If \( \sigma_i \) are the singular values of \( R \), then the nuclear norm is the sum of those singular values. The objective of this model is to balance the minimization of the approximation errors in the known entries and the nuclear norm of \( R \):

$$
\min_{R \in \mathbb{R}^{m \times n}} \frac{1}{2} \sum_{(u,i) \in \Delta} (R_{ui} - A_{ui})^2 + \lambda_3 \left\| R \right\|_\ast
$$

Where:

- $ R \in \mathbb{R}^{m \times n} $ is the full constructed matrix  
- $ \left\| R \right\|_\ast = \sum_i \sigma_i(R) $ is the **nuclear norm**, or the sum of the singular values of $ R $  
- $ \lambda_3 > 0 $ is the regularization parameter controlling the weight of the nuclear norm

---

Unlike the $ \ell_2 $-regularized matrix factorization model, this formulation does not factor the matrix into $ X $ and $ Y $; instead, it directly optimizes over the matrix $ R $, penalizing its nuclear norm to encourage **low-rank solutions**.

The nuclear norm serves as a **convex relaxation of the matrix rank function**, making this optimization problem convex and solvable using standard convex programming techniques. Overall, this model seeks to:

- Minimize squared reconstruction error on known ratings
- Promote a low-rank structure in the completed matrix via nuclear norm regularization

In [10]:
# nuclear norm model, using dataset 1
nuclear_matrix_1 = nuclear_norm_model_df(df_1)
recommend_anime(nuclear_matrix_1, chosen_user_1, df_1)

Optimization succeeded.


,title,predicted_rating
0,Clannad: After Story,9.208856
1,Ookami Kodomo no Ame to Yuki,9.038716
2,Azumanga Daioh,8.990107
3,Psycho-Pass,8.780612
4,Nichijou,8.421423


In [11]:
# nuclear norm model, using dataset 2
nuclear_matrix_2 = nuclear_norm_model_df(df_2)
recommend_anime(nuclear_matrix_2, chosen_user_2, df_2)

Optimization succeeded.


,title,predicted_rating
0,Gekkan Shoujo Nozaki-kun,8.969728
1,Yuri!!! on Ice,8.940827
2,Sakura Quest,8.543998
3,Mob Psycho 100 II,8.517210
4,Asobi Asobase,8.290307
